In [51]:

import random
# Import the Python SDK
import google.generativeai as genai # type: ignore
import json
import os

# Load the JSON file
config_file = "config.json"  # Replace with your JSON file's name if different
if os.path.exists(config_file):
    with open(config_file, "r") as file:
        config = json.load(file)
        GEMINI_API_KEY = config.get("GEMINI_API_KEY")
else:
    raise FileNotFoundError(f"Configuration file '{config_file}' not found.")

genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel('gemini-pro')

# HTML şablonunu yüklemek için fonksiyon
def load_template(template_path, context):
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"Template file '{template_path}' not found.")
    
    with open(template_path, 'r', encoding='utf-8') as template_file:
        template = template_file.read()
        for key, value in context.items():
            template = template.replace(f"{{{{ {key} }}}}", value)
    return template

# Phishing parametreleri
phishing_examples = [
    {"Reason": "Security Alert: Unusual Login Attempt", "Fake Link": "https://logincheck.microsoftsecure.com", 
     "Created By": "Alex Phish", "Company": "Proximus", "Department": "IT Security"},
    #{"Reason": "Urgent Action Required: Password Expiration", "Fake Link": "https://passwordreset.googleapps.com", 
     #"Created By": "Sara Spoof", "Company": "Google", "Department": "Account Management"},
    {"Reason": "Security_alert: Enable Two-Factor Authentication.", "Fake Link": "https://hr-portal.companybenefits.com", 
     "Created By": "Ben Benefits", "Company": "Proximus", "Department": "HR"},
    #{"Reason": "Cloud Storage Limit Reached", "Fake Link": "https://cloudsync.dropprovider.net", 
     #"Created By": "Clara Cloud", "Company": "Dropbox", "Department": "Support"},
    {"Reason": "Package_delivery:Tomorrow we will deliver a package to you!", "Fake Link": "https://update-software.adobecheck.com", 
     "Created By": "Andy Update", "Company": "BPost", "Department": "Delivery Department"},
    {"Reason": "Website Issue", "Fake Link": "https://secureaccount.bankalerts.net", 
     "Created By": "Nancy Notice", "Company": "Proximus", "Department": "IT Support"},
    #{"Reason": "Account_issue: Irregularities on your OneDrive account", "Fake Link": "https://register-now.eventwebinar.net", 
     #"Created By": "Tina Tech", "Company": "Tech Webinars Inc.", "Department": "Events Team"},
    #{"Reason": "Claim Your Free Subscription", "Fake Link": "https://offers.freelicensepro.com", 
     #"Created By": "Sam Freebie", "Company": "Premium Software Deals", "Department": "Marketing"},
    {"Reason": "Verify Your Account Information", "Fake Link": "https://secureverify.amazon-support.net", 
     "Created By": "Liam Check", "Company": "Amazon", "Department": "Customer Verification"},
    #{"Reason": "Suspicious Activity Detected in Your Account", "Fake Link": "https://alert-check.netflixprotect.com", 
    # "Created By": "Emily Alert", "Company": "Netflix", "Department": "Security Team"},
    #{"Reason": "Exclusive Discount for Our Corporate Partners", "Fake Link": "https://partnerspecials.samsungoffers.net", 
     #"Created By": "Mark Deals", "Company": "Samsung", "Department": "Corporate Sales"},
    {"Reason": "Important: Confirm Your Delivery Address", "Fake Link": "https://deliveryconfirmation.ups-tracking.net", 
     "Created By": "Chris Courier", "Company": "BPost", "Department": "Logistics"}
]


# Çalışanlar listesini json dosyasından okuma
employees = []
with open('employees.json', mode='r') as file:
    employees = json.load(file)  # JSON dosyasını doğrudan listeye yükle

# Şirket ve şablon eşleştirmesi
company_templates = {
    "Proximus": "templates/proximus.html",
    "Amazon": "templates/amazon.html",
    "BPost": "templates/bpost.html"
    # Diğer şirketler için şablon yollarını ekleyin...
}

# Tüm çalışanlara e-posta gönderme
processed_emails = [] 
for employee in employees:
    random_pick = random.choice(phishing_examples)
    
    # Prompt oluşturma
    prompt = f'''Write a professional email for a phishing scenario. 
    The email should appear to come from {random_pick["Created By"]}, 
    representing the company {random_pick["Company"]} and department {random_pick["Department"]}. 
    The email is addressed to an employee of Proximus and should focus on the following subject: {random_pick["Reason"]}.

    The email must:

    1. Be persuasive and appear legitimate.
    2. Contain a clear call to action directing the recipient to click the link: {random_pick["Fake Link"]}.
    3. Use polite and formal language consistent with the company's communication style.
    4. Avoid any direct mention or hint that this is a phishing attempt.

    Format the email as follows:

    - Subject: {random_pick["Reason"]}
    - Greeting: Start with "Dear {employee["name"]} {employee["surname"]},"
    - Body: Briefly explain the situation, provide a reason for urgency, and introduce the provided link.Put the link inside an <a> tag that would be clickable.
    - Closing: Use a professional closing, such as "Best regards," followed by {random_pick["Created By"]} and their department.

    Ensure the email content is realistic, convincing, and free of grammatical errors.'''

    try:
        # Metin oluşturma
        response = model.generate_content(prompt)
        print(response.text)

        # Şablon için bağlam (context)
        context = {
            "subject": random_pick["Reason"],
            "recipient_name": f"{employee['name']} {employee['surname']}",
            "body_content": response.text,
            "fake_link": random_pick["Fake Link"],
            "sender_name": random_pick["Created By"],
            "department": random_pick["Department"]
        }
     # Tüm içeriği birleştirip tek bir string oluştur
        context["full_content"] = f"""
        <h1>{context['subject']}</h1>
        <p>Dear {context['recipient_name']},</p>
        <p>{context['body_content']}</p>
         <p>Click here: <a href="{context['fake_link']}">{context['fake_link']}</a></p>
        <p>Best regards,<br>{context['sender_name']}<br>{context['department']}</p>
         """
        
        # Şirket adını almak ve uygun şablonu belirlemek
        company_name = random_pick["Company"]
        print(company_name)
        template_path = company_templates.get(company_name)
        print(template_path)

        if not template_path:
            print(f"No template found for company '{company_name}'. Skipping this email.")
            continue  # Eğer şablon yoksa, bu e-postayı atla

        email_body = load_template(template_path, {"full_content": response.text})

        processed_emails.append({
            "to": employee["email"],
            "subject": random_pick["Reason"],
            "body": email_body
        })

    except Exception as e:
        print(f"Error generating email for {employee['name']} {employee['surname']}: {e}")
    
    
#print(body.text)

# E-posta sonuçlarını HTML dosyasına yazma
    with open('sample_emails/'+employee['name']+'.html', 'w', encoding='utf-8') as f:
        f.write('''<!DOCTYPE html>
        <html>
        <head><title></title></head>
        <body>
        <h1></h1>
        <ul>
        ''')

        f.write(f'''
            <li>
                <strong>To:</strong> {employee["email"]}<br>
                <strong>Subject:</strong> {random_pick["Reason"]}<br>
                <strong>Body:</strong><br>
                <pre>{email_body}</pre>
                <hr>
            </li>
        ''')

        f.write('''</ul></body></html>''')


**Subject: Website Issue**

Dear Yeliz Kaya,

I hope this email finds you well.

I'm writing to inform you about a temporary issue with the Proximus website that requires urgent attention. Our IT support team has identified an error that may affect the availability and functionality of the website. 

To resolve this issue, it's crucial that you take immediate action by clicking the following link:

[<a href="https://secureaccount.bankalerts.net">Click Here</a>]

By clicking the link, you will be directed to a secure page where you can verify your account details and resolve the issue. This process should take no more than a few minutes.

Please complete this process as soon as possible to ensure the smooth operation of the website.

Thank you for your cooperation and understanding.

Best regards,
Nancy Notice
IT Support
Proximus
Proximus
templates/proximus.html
**Subject: Security_alert: Enable Two-Factor Authentication**

**Dear Anastasia Smith,**

Due to recent security concerns, Pro

We prompt Gemini for the subject, e.g. we feed it previous output and ask for the subject:

In [24]:
subject = model.generate_content("Write me the subject of this email:\n"+body.text)
print(subject.text)

Password Reset Notice


Sending an email using your gmail account.

In [25]:
from google.auth.transport.requests import Request # type: ignore
from google.oauth2.credentials import Credentials # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow # type: ignore
from googleapiclient.discovery import build # type: ignore
from googleapiclient.errors import HttpError # type: ignore

# If modifying these scopes, delete the file token.json.
SCOPES = [
    "https://www.googleapis.com/auth/gmail.send",
]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

ModuleNotFoundError: No module named 'google_auth_oauthlib'

In [10]:
# Gerekli kütüphanelerin import edilmesi
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# E-posta ayarları
smtp_server = "smtp.gmail.com"  # Örnek SMTP sunucusu
smtp_port = 587  # SMTP portu
sender_email = "your_email@gmail.com"  # Gönderen e-posta
password = "your_email_password"  # Gönderen e-posta şifresi
recipient_email = "employee_email@example.com"  # Alıcının e-posta adresi

# E-posta içeriği
subject = "Hesap Güvenliğiniz için Önemli Bir Bildirim"
html_content = """
<html>
<head>
    <style>
        body { font-family: Arial, sans-serif; }
        .header { background-color: #0073e6; color: white; padding: 10px; }
        .content { margin: 20px; }
    </style>
</head>
<body>
    <div class="header">
        <h1>Proximus</h1>
    </div>
    <div class="content">
        <h2>Hesap Aktivitesi Uyarısı</h2>
        <p>Hesabınızda şüpheli bir aktivite tespit edildi. Lütfen hemen aşağıdaki bağlantıya tıklayarak güvenlik adımlarını takip edin:</p>
        <a href="https://example.com/secure-login">Hesabınızı Güvenli Hale Getirin</a>
        <p>Teşekkürler,<br>Proximus Güvenlik Ekibi</p>
    </div>
</body>
</html>
"""

# E-posta mesajını oluştur
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = recipient_email
msg['Subject'] = subject
msg.attach(MIMEText(html_content, 'html'))

# E-posta gönderimi
try:
    # SMTP sunucusuna bağlan
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # TLS bağlantısını başlat
        server.login(sender_email, password)  # Giriş yap
        server.sendmail(sender_email, recipient_email, msg.as_string())  # E-postayı gönder
        print("E-posta başarıyla gönderildi!")
except Exception as e:
    print(f"E-posta gönderiminde hata: {e}")


E-posta gönderiminde hata: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 5b1f17b1804b1-438b31a1e7fsm57693435e9.10 - gsmtp')


In [ ]:
phishing_examples = [
    {"Reason": "Security Alert: Unusual Login Attempt", "Fake Link": "https://logincheck.microsoftsecure.com", 
     "Created By": "Alex Phish", "Company": "Proximus", "Department": "IT Security"},
  
    {"Reason": "Security_alert: Enable Two-Factor Authentication.", "Fake Link": "https://hr-portal.companybenefits.com", 
     "Created By": "Ben Benefits", "Company": "Proximus", "Department": "HR"},
   
    {"Reason": "Package_delivery:Tomorrow we will deliver a package to you!", "Fake Link": "https://update-software.adobecheck.com", 
     "Created By": "Andy Update", "Company": "BPost", "Department": "Delivery Department"},

    {"Reason": "Website Issue", "Fake Link": "https://secureaccount.bankalerts.net", 
     "Created By": "Nancy Notice", "Company": "Proximus", "Department": "IT Support"},
   
    {"Reason": "Verify Your Account Information", "Fake Link": "https://secureverify.amazon-support.net", 
     "Created By": "Liam Check", "Company": "Amazon", "Department": "Customer Verification"},


    {"Reason": "Important: Confirm Your Delivery Address", "Fake Link": "https://deliveryconfirmation.ups-tracking.net", 
     "Created By": "Chris Courier", "Company": "BPost", "Department": "Logistics"}
]


Then we create the necessary functions: